In [1]:
# Import các thư viện cần thiết
import os
import cv2  # Để load và xử lý video
import mediapipe as mp  # Pose estimation (lightweight, real-time)
from ultralytics import YOLO  # YOLOv8 cho club/ball detection (cần install ultralytics nếu chưa có)
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd  # Để quản lý metadata/labels
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score  # Evaluation metrics
import time  # Để đo FPS
import json  # Để lưu keypoints nếu cần

# Khởi tạo MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils  # Để vẽ skeleton nếu cần visualize
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5)

# Khởi tạo YOLOv8 model (giả sử dùng pretrained, hoặc train sau)
yolo_model = YOLO('yolov8n.pt')  # Nano version cho lightweight; thay bằng path model trained nếu có

In [2]:
# Cell 2: Data Loading và Preprocessing (ĐÃ SỬA LỖI - CHẠY LẠI CELL NÀY THÔI)

import os
from sklearn.model_selection import train_test_split

def load_dataset():
    # Tự động tìm thư mục Public Test ở các nơi phổ biến
    possible_paths = [
        'Public Test',
        './Public Test',
        '../Public Test',
        'D:/Public Test',
        'C:/Public Test',
        os.path.join(os.path.expanduser('~'), 'Desktop', 'Public Test'),
        os.path.join(os.path.expanduser('~'), 'Downloads', 'Public Test'),
        os.path.join(os.path.expanduser('~'), 'OneDrive', 'Public Test'),
    ]
    
    root_dir = None
    for path in possible_paths:
        if os.path.exists(path):
            root_dir = path
            break
    
    if root_dir is None:
        print("Không tìm thấy thư mục 'Public Test' ở các nơi phổ biến.")
        print("Bạn hãy làm 1 trong 2 cách sau:\n")
        print("CÁCH 1 (dễ nhất):")
        print("   - Mở File Explorer")
        print("   - Tìm thư mục tên 'Public Test'")
        print("   - Click chuột phải vào thư mục đó → Copy path")
        print("   - Paste vào ô dưới đây và chạy lại cell này\n")
        root_dir = input("Dán đường dẫn đầy đủ vào đây rồi nhấn Enter: ").strip().strip('"')
        if not os.path.exists(root_dir):
            print("Đường dẫn vẫn sai. Cell này sẽ trả về dữ liệu rỗng để tránh lỗi.")
            return [], [], []

    print(f"Đã tìm thấy dữ liệu tại: {root_dir}")
    
    data = []      # List of video frames
    labels = []    # 0: bad, 1: good
    metadata = []  # Thông tin chi tiết
    
    environments = ['Indoor', 'Outdoor']
    for env in environments:
        env_dir = os.path.join(root_dir, env)
        if not os.path.isdir(env_dir):
            continue
        for band in os.listdir(env_dir):
            band_dir = os.path.join(env_dir, band)
            if not os.path.isdir(band_dir):
                continue
            for vid_file in os.listdir(band_dir):
                if vid_file.lower().endswith('.mov'):
                    vid_path = os.path.join(band_dir, vid_file)
                    
                    # Gán nhãn theo band
                    if '1-2' in band:
                        label_class = 0
                        score = 1.5
                    elif '2-4' in band:
                        label_class = 0
                        score = 3
                    elif '4-6' in band:
                        label_class = 0
                        score = 5
                    elif '6-8' in band:
                        label_class = 1
                        score = 7
                    elif '8-10' in band:
                        label_class = 1
                        score = 9
                    else:
                        continue  # Bỏ qua band lạ
                    
                    # Đọc video
                    cap = cv2.VideoCapture(vid_path)
                    if not cap.isOpened():
                        continue
                    frames = []
                    frame_count = 0
                    fps = cap.get(cv2.CAP_PROP_FPS)
                    downsample_rate = max(1, int(fps / 30))
                    while True:
                        ret, frame = cap.read()
                        if not ret:
                            break
                        if frame_count % downsample_rate == 0:
                            frames.append(frame)
                        frame_count += 1
                    cap.release()
                    
                    if len(frames) > 10:  # Chỉ giữ video có ít nhất 10 frames
                        data.append(frames)
                        labels.append(label_class)
                        metadata.append({'env': env, 'band': band, 'path': vid_path, 'score': score, 'num_frames': len(frames)})
    
    return data, labels, metadata

# CHẠY DÒNG NÀY ĐỂ LOAD DỮ LIỆU
videos, labels, metadata = load_dataset()

if len(videos) == 0:
    print("\nChưa load được video nào. Bạn kiểm tra lại đường dẫn nhé!")
    print("Sau khi sửa đường dẫn và chạy lại cell này, tiếp tục chạy Cell 3 bình thường.")
else:
    print(f"Thành công! Đã load {len(videos)} video")
    df = pd.DataFrame(metadata)
    print("Chi tiết:")
    print(df[['env', 'band', 'score', 'num_frames']])
    
    # Chia train/test (an toàn, không còn lỗi)
    train_vids, test_vids, train_labels, test_labels = train_test_split(
        videos, labels, test_size=0.2, random_state=42, stratify=labels)
    
    # Chia metadata tương ứng
    train_idx, test_idx = train_test_split(range(len(metadata)), test_size=0.2, random_state=42, stratify=labels)
    train_metadata = [metadata[i] for i in train_idx]
    test_metadata = [metadata[i] for i in test_idx]
    
    print(f"Chia xong: {len(train_vids)} video train, {len(test_vids)} video test")

Đã tìm thấy dữ liệu tại: Public Test
Thành công! Đã load 50 video
Chi tiết:
        env       band  score  num_frames
0    Indoor   Band 1-2    1.5         325
1    Indoor   Band 1-2    1.5         311
2    Indoor   Band 1-2    1.5         306
3    Indoor   Band 1-2    1.5         240
4    Indoor   Band 1-2    1.5         342
5    Indoor   Band 1-2    1.5         241
6    Indoor   Band 2-4    3.0         280
7    Indoor   Band 2-4    3.0         257
8    Indoor   Band 2-4    3.0         283
9    Indoor   Band 2-4    3.0         246
10   Indoor   Band 4-6    5.0         288
11   Indoor   Band 4-6    5.0         291
12   Indoor   Band 4-6    5.0         329
13   Indoor   Band 4-6    5.0         199
14   Indoor   Band 4-6    5.0         216
15   Indoor   Band 4-6    5.0         217
16   Indoor   Band 6-8    7.0         221
17   Indoor   Band 6-8    7.0         222
18   Indoor   Band 6-8    7.0         203
19   Indoor   Band 6-8    7.0         269
20   Indoor   Band 6-8    7.0         239


In [4]:
# Hàm extract keypoints từ frames dùng MediaPipe
def extract_keypoints(frames):
    keypoints_seq = []
    for frame in frames:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)
        if results.pose_landmarks:
            keypoints = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten()  # 33 landmarks * 3 = 99
            keypoints_seq.append(keypoints)
        else:
            keypoints_seq.append(np.zeros(99))  # Padding nếu không detect
    return np.array(keypoints_seq)

# Extract cho train và test
keypoints_train = [extract_keypoints(v) for v in train_vids]
keypoints_test = [extract_keypoints(v) for v in test_vids]

# Lưu nếu cần (ví dụ: JSON)
# for i, kp in enumerate(keypoints_train):
#     with open(f'train_kp_{i}.json', 'w') as f:
#         json.dump(kp.tolist(), f)

# Hàm tính metrics từ keypoints (ví dụ: hip rotation angle)
def calculate_rotation(keypoints):  # Keypoints là per frame
    # Ví dụ đơn giản: angle giữa hip left-right (landmark 23-24)
    if keypoints.shape[1] < 99:
        return 0
    hip_left = keypoints[:, 23*3:23*3+3]  # x,y,z
    hip_right = keypoints[:, 24*3:24*3+3]
    diff = hip_left - hip_right
    angle = np.arctan2(diff[:,1], diff[:,0]) * 180 / np.pi  # Độ
    return np.mean(angle)  # Average rotation

In [5]:
print(keypoints_train[0].shape)  # Nên là (num_frames, 99) ví dụ (325, 99)

(247, 99)


In [6]:
# Cell 4: SKIP Object Detection - Ước lượng club angle & impact timing từ MediaPipe keypoints

import numpy as np

# Các landmark index từ MediaPipe Pose (COCO format)
# 11: left_shoulder, 12: right_shoulder
# 13: left_elbow, 14: right_elbow
# 15: left_wrist, 16: right_wrist
# 23: left_hip, 24: right_hip
# 25: left_knee, 26: right_knee

def calculate_club_proxy_angle(keypoints_per_frame):
    """
    Ước lượng club angle từ wrist (giả sử tay lead là left cho right-handed golfer)
    Trả về angle (độ) giữa vector elbow->wrist so với horizontal.
    """
    if keypoints_per_frame.shape[0] != 99:  # 33*3
        return 0.0
    
    # Reshape thành (33, 3) [x,y,z] normalized
    kp = keypoints_per_frame.reshape(33, 3)
    
    # Giả sử right-handed golfer: dùng right wrist (16) cho trail hand, left (15) cho lead
    # Simple: vector từ left_elbow (13) -> left_wrist (15)
    elbow = kp[13, :2]  # x,y
    wrist = kp[15, :2]
    vector = wrist - elbow
    
    # Angle với horizontal (x-axis)
    angle = np.arctan2(vector[1], vector[0]) * 180 / np.pi
    return angle  # Có thể negative, dùng abs nếu cần

def estimate_impact_frame(keypoints_sequence):
    """
    Ước lượng frame impact: nơi wrist velocity max ở downswing (từ backswing -> forward)
    Trả về index frame gần impact nhất (hoặc None nếu không detect)
    """
    if len(keypoints_sequence) < 10:
        return None
    
    # Tính velocity của right wrist (trail hand)
    wrists = keypoints_sequence[:, 16*3:16*3+2]  # x,y của wrist qua các frame
    velocities = np.linalg.norm(np.diff(wrists, axis=0), axis=1)  # speed per frame
    
    # Impact ~ max velocity ở nửa sau swing (giả sử backswing đầu, downswing sau)
    mid = len(velocities) // 2
    impact_idx = mid + np.argmax(velocities[mid:])  # max vel ở nửa sau
    return impact_idx + 1  # +1 vì diff mất 1 frame

# Apply cho train/test keypoints (từ Cell 3)
print("Đang tính club proxy angle & impact estimation cho train...")
train_club_angles = []
train_impact_frames = []
for kp_seq in keypoints_train:
    angles = [calculate_club_proxy_angle(frame) for frame in kp_seq]
    train_club_angles.append(np.mean(angles))  # Average hoặc list full
    impact = estimate_impact_frame(kp_seq)
    train_impact_frames.append(impact)

print("Đang tính cho test...")
test_club_angles = []
test_impact_frames = []
for kp_seq in keypoints_test:
    angles = [calculate_club_proxy_angle(frame) for frame in kp_seq]
    test_club_angles.append(np.mean(angles))
    impact = estimate_impact_frame(kp_seq)
    test_impact_frames.append(impact)

# Ví dụ in ra
if test_club_angles:
    print(f"Test video 0: Average club proxy angle ~ {test_club_angles[0]:.2f} độ")
    print(f"Estimated impact frame: {test_impact_frames[0]} / {len(keypoints_test[0])}")

print("Hoàn tất ước lượng từ keypoints! Bỏ qua YOLO/Roboflow.")

Đang tính club proxy angle & impact estimation cho train...
Đang tính cho test...
Test video 0: Average club proxy angle ~ 45.48 độ
Estimated impact frame: 160 / 257
Hoàn tất ước lượng từ keypoints! Bỏ qua YOLO/Roboflow.


In [7]:
# Định nghĩa 8 phases: 0: Address, 1: Takeaway, 2: Backswing, 3: Top, 4: Downswing, 5: Impact, 6: Follow Through, 7: Finish
# Giả sử bạn có phase_labels từ annotation (array per video: num_frames x 1 với 0-7)
# Nếu chưa, cần annotate thủ công dùng CVAT/Roboflow

# Model LSTM
class SwingPhaseLSTM(nn.Module):
    def __init__(self, input_size=99, hidden_size=128, num_layers=2, num_classes=8):
        super(SwingPhaseLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # Bidirectional

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)  # (batch, seq_len, num_classes)
        return out

# Instance model
phase_model = SwingPhaseLSTM()
optimizer = torch.optim.Adam(phase_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Dataset class
class GolfDataset(Dataset):
    def __init__(self, keypoints, phase_labels):
        self.keypoints = [torch.tensor(kp, dtype=torch.float32) for kp in keypoints]
        self.phase_labels = [torch.tensor(pl, dtype=torch.long) for pl in phase_labels]  # Giả sử phase_labels là list of arrays

    def __len__(self):
        return len(self.keypoints)

    def __getitem__(self, idx):
        return self.keypoints[idx], self.phase_labels[idx]

# Giả sử phase_labels_train/test từ annotation
# phase_labels_train = [np.random.randint(0,8, size=len(kp)) for kp in keypoints_train]  # Placeholder
# train_dataset = GolfDataset(keypoints_train, phase_labels_train)
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: (torch.nn.utils.rnn.pad_sequence([i[0] for i in x], batch_first=True),
#                                                                                         torch.nn.utils.rnn.pad_sequence([i[1] for i in x], batch_first=True, padding_value=-1)))

# Training loop (comment out nếu chưa có labels)
# for epoch in range(20):
#     phase_model.train()
#     total_loss = 0
#     for keypoints, labels in train_loader:
#         outputs = phase_model(keypoints)
#         mask = labels != -1  # Ignore padding
#         loss = criterion(outputs[mask], labels[mask])
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f'Epoch {epoch+1}: Loss {total_loss / len(train_loader)}')

# Inference example
def predict_phases(keypoints_seq):
    phase_model.eval()
    with torch.no_grad():
        inputs = torch.tensor(keypoints_seq, dtype=torch.float32).unsqueeze(0)  # (1, seq_len, 99)
        outputs = phase_model(inputs)
        phases = torch.argmax(outputs, dim=-1).squeeze(0).numpy()  # Per frame phases
    return phases

In [8]:
# Hàm get embedding từ keypoints sequence (simple average hoặc LSTM hidden)
def get_embedding(keypoints_seq):
    return np.mean(keypoints_seq, axis=0)  # (99,)

# Giả sử professional reference embedding (từ GolfDB hoặc pro videos)
pro_embedding = np.random.randn(99)  # Placeholder; load từ file thực tế np.load('pro_embedding.npy')

# Similarity scoring
def score_swing(embedding, pro_embedding):
    cosine_sim = np.dot(embedding, pro_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(pro_embedding))
    return cosine_sim

# Classification (good/bad dựa trên sim > threshold)
def classify_swing(sim_score, threshold=0.7):
    return 1 if sim_score > threshold else 0  # Good/Bad

# Generate feedback dựa trên phases và metrics
def generate_feedback(phases, rotation, club_angle):
    issues = []
    if np.max(phases == 2):  # Backswing issue example
        if rotation > 15:
            issues.append("Over-rotation in backswing (>15°).")
    if club_angle > 10:  # Placeholder
        issues.append("Early release in downswing.")
    if issues:
        return "Needs Improvement: " + "; ".join(issues)
    return "Good Swing!"

# Example trên test set
preds = []
feedbacks = []
for i, kp in enumerate(keypoints_test):
    phases = predict_phases(kp)
    rotation = calculate_rotation(kp)
    club_angle = 0  # Từ detection
    emb = get_embedding(kp)
    sim = score_swing(emb, pro_embedding)
    pred = classify_swing(sim)
    fb = generate_feedback(phases, rotation, club_angle)
    preds.append(pred)
    feedbacks.append(fb)

print(f'Accuracy: {accuracy_score(test_labels, preds)}')
print(f'F1: {f1_score(test_labels, preds)}')

Accuracy: 0.6
F1: 0.0


In [10]:
# Cell 4: Biomechanical Features Extraction từ Keypoints (chỉ dùng body pose)
# Không detect club/ball → dùng proxy angles từ wrists, hips, shoulders

import numpy as np

# Landmark indices MediaPipe Pose (33 keypoints, 0-based)
# 11: left_shoulder, 12: right_shoulder
# 13: left_elbow, 14: right_elbow
# 15: left_wrist, 16: right_wrist
# 23: left_hip, 24: right_hip
# 25: left_knee, 26: right_knee
# 27: left_ankle, 28: right_ankle

def extract_biomechanical_features(keypoints_seq):
    """
    keypoints_seq: (num_frames, 99)  # flattened 33*3 (x,y,z)
    Trả về dict features: angles, velocities, etc. per frame + summary
    """
    if len(keypoints_seq) == 0:
        return {}
    
    kp = keypoints_seq.reshape(-1, 33, 3)  # (frames, 33, 3) [x,y,z]
    
    features = {}
    
    # 1. Hip rotation (X-factor proxy: angle giữa hips so với shoulders)
    hip_left = kp[:, 23, :2]   # x,y
    hip_right = kp[:, 24, :2]
    shoulder_left = kp[:, 11, :2]
    shoulder_right = kp[:, 12, :2]
    
    hip_vector = hip_left - hip_right
    shoulder_vector = shoulder_left - shoulder_right
    
    hip_angle = np.arctan2(hip_vector[:,1], hip_vector[:,0]) * 180 / np.pi
    shoulder_angle = np.arctan2(shoulder_vector[:,1], shoulder_vector[:,0]) * 180 / np.pi
    features['x_factor'] = np.abs(shoulder_angle - hip_angle)  # Độ lệch rotation
    
    # 2. Lead arm angle (left elbow for right-handed golfer)
    elbow_left = kp[:, 13, :2]
    wrist_left = kp[:, 15, :2]
    arm_vector = wrist_left - elbow_left
    arm_angle = np.arctan2(arm_vector[:,1], arm_vector[:,0]) * 180 / np.pi
    features['lead_arm_angle'] = arm_angle
    
    # 3. Wrist velocity (proxy cho club speed/impact timing)
    wrist_right = kp[:, 16, :2]  # trail wrist
    wrist_vel = np.linalg.norm(np.diff(wrist_right, axis=0), axis=1)
    features['wrist_velocity'] = np.concatenate([[0], wrist_vel])  # pad frame 0
    
    # 4. Sway (hip movement horizontal)
    hip_center = (hip_left + hip_right) / 2
    hip_sway = np.std(hip_center[:,0])  # variance x-coordinate
    
    # Summary stats
    features['max_x_factor'] = np.max(features['x_factor'])
    features['max_wrist_vel'] = np.max(features['wrist_velocity'])
    features['avg_sway'] = hip_sway
    
    return features

# Apply cho tất cả train/test
print("Extracting biomechanical features cho train...")
train_features = [extract_biomechanical_features(kp) for kp in keypoints_train]

print("Extracting cho test...")
test_features = [extract_biomechanical_features(kp) for kp in keypoints_test]

# Ví dụ in summary cho test video 0
if test_features:
    print("Test video 0 biomechanical summary:")
    print(f"  Max X-Factor (rotation diff): {test_features[0]['max_x_factor']:.2f}°")
    print(f"  Max Wrist Velocity (proxy club speed): {test_features[0]['max_wrist_vel']:.4f}")
    print(f"  Avg Hip Sway: {test_features[0]['avg_sway']:.4f}")

Extracting biomechanical features cho train...
Extracting cho test...
Test video 0 biomechanical summary:
  Max X-Factor (rotation diff): 356.79°
  Max Wrist Velocity (proxy club speed): 0.6263
  Avg Hip Sway: 0.0200


In [12]:
# Cell 5: Temporal Model - LSTM cho Phase Detection & Swing Classification
# Input: keypoints sequence + biomechanical features
# Output: per-frame phase (8 classes) + overall good/bad

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Giả sử bạn chưa có phase labels thật → dùng rule-based để tạo pseudo-labels (dựa trên velocity & angles)
def generate_pseudo_phase_labels(keypoints_seq, features):
    """
    Rule-based phase detection đơn giản dựa trên biomechanics:
    - Address: frame đầu, wrist vel thấp
    - Takeaway: wrist vel tăng nhẹ
    - Backswing: wrist lên cao, x-factor tăng
    - Top: wrist vel = 0, max arm angle
    - Downswing: wrist vel tăng mạnh
    - Impact: max wrist vel
    - Follow: vel giảm
    - Finish: vel ~0
    """
    num_frames = len(keypoints_seq)
    labels = np.zeros(num_frames, dtype=int)
    
    wrist_vel = features['wrist_velocity']
    x_factor = features['x_factor']
    arm_angle = features['lead_arm_angle']
    
    # Tìm các điểm mốc
    impact_idx = np.argmax(wrist_vel) if np.any(wrist_vel > 0.01) else num_frames // 2
    top_idx = np.argmax(np.abs(arm_angle)) if np.any(np.abs(arm_angle) > 30) else impact_idx // 2
    
    # Gán phases (rule heuristic)
    labels[:max(1, top_idx//3)] = 0     # Address / early takeaway
    labels[max(1, top_idx//3):top_idx] = 1  # Mid backswing
    labels[top_idx] = 3                 # Top
    labels[top_idx+1:impact_idx] = 4    # Downswing
    labels[impact_idx] = 5              # Impact
    labels[impact_idx+1:impact_idx + (num_frames - impact_idx)//2] = 6  # Early follow
    labels[impact_idx + (num_frames - impact_idx)//2:] = 7  # Finish
    
    return labels

# Tạo pseudo labels cho train
phase_labels_train = [generate_pseudo_phase_labels(kp, feat) for kp, feat in zip(keypoints_train, train_features)]

# Model LSTM
class GolfPhaseLSTM(nn.Module):
    def __init__(self, input_size=99 + 5, hidden_size=128, num_classes=8):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, bidirectional=True)
        self.fc_phase = nn.Linear(hidden_size * 2, num_classes)          # 256 → 8
        self.fc_class = nn.Linear(hidden_size * 2, 1)                    # 256 → 1

    def forward(self, x):
        out, (h, c) = self.lstm(x)  # out: (batch, seq, 256), h: (4, batch, 128)
        
        # Phase prediction: dùng toàn bộ output sequence
        phase_out = self.fc_phase(out)  # (batch, seq_len, 8)
        
        # Classification: dùng hidden state cuối của bidirectional (concat forward + backward)
        # h shape: (num_layers*2, batch, hidden) = (4, batch, 128)
        # Lấy 2 hidden cuối (layer 2 forward + backward)
        last_hidden = torch.cat((h[-2], h[-1]), dim=-1)  # (batch, 128 + 128) = (batch, 256)
        class_out = torch.sigmoid(self.fc_class(last_hidden))  # (batch, 1)
        
        return phase_out, class_out.squeeze(-1)  # squeeze để thành (batch,)

model = GolfPhaseLSTM()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion_phase = nn.CrossEntropyLoss()
criterion_class = nn.BCELoss()

# Dataset (pad sequences vì length khác nhau)
class GolfBioDataset(Dataset):
    def __init__(self, keypoints_list, phase_labels_list, features_list):
        self.data = []
        for kp, pl, feat in zip(keypoints_list, phase_labels_list, features_list):
            # Concat extra bio features (repeat per frame)
            extra = np.stack([feat['x_factor'], feat['lead_arm_angle'], feat['wrist_velocity'],
                              feat['x_factor']*0.5, feat['lead_arm_angle']*0.5], axis=1)  # 5 features
            seq = np.concatenate([kp, extra], axis=1)  # (frames, 99+5)
            self.data.append((torch.tensor(seq, dtype=torch.float32), torch.tensor(pl, dtype=torch.long)))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Pad collate
def pad_collate(batch):
    seqs, labels = zip(*batch)
    seqs_pad = torch.nn.utils.rnn.pad_sequence(seqs, batch_first=True)
    labels_pad = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)  # ignore_index
    return seqs_pad, labels_pad

train_dataset = GolfBioDataset(keypoints_train, phase_labels_train, train_features)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=pad_collate)

# Training loop (chạy 10-20 epochs)
# Training loop
for epoch in range(15):
    model.train()
    total_loss = 0
    for seqs, phase_labels in train_loader:  # seqs: padded keypoints+features, phase_labels: padded phases
        phase_out, class_out = model(seqs)
        
        # Phase loss
        mask = phase_labels != -100
        phase_loss = criterion_phase(phase_out[mask], phase_labels[mask])
        
        # Class loss: dùng train_labels binary (0/1) cho batch này
        # Giả sử batch size nhỏ, lấy labels tương ứng (cần index nếu shuffle)
        # Để đơn giản tạm thời: skip class loss hoặc dùng pseudo
        # Hoặc: thêm class_labels vào Dataset và loader
        
        # Tạm chỉ train phase (sau fix class)
        loss = phase_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1:2d}: Avg Loss {avg_loss:.4f}")
print("Training xong! Model phase detection dựa thuần biomechanical keypoints.")

Epoch  1: Avg Loss 1.6725
Epoch  2: Avg Loss 1.1820
Epoch  3: Avg Loss 0.9749
Epoch  4: Avg Loss 0.8339
Epoch  5: Avg Loss 0.7885
Epoch  6: Avg Loss 0.6735
Epoch  7: Avg Loss 0.6080
Epoch  8: Avg Loss 0.5903
Epoch  9: Avg Loss 0.6503
Epoch 10: Avg Loss 0.5804
Epoch 11: Avg Loss 0.5930
Epoch 12: Avg Loss 0.5142
Epoch 13: Avg Loss 0.4761
Epoch 14: Avg Loss 0.4338
Epoch 15: Avg Loss 0.4547
Training xong! Model phase detection dựa thuần biomechanical keypoints.


In [14]:
# === INFERENCE: Dự đoán phases cho test set ===
model.eval()
test_phase_preds = []

with torch.no_grad():
    for i, kp_seq in enumerate(keypoints_test):
        # Tái tạo input giống training: keypoints + extra bio features
        feat = test_features[i]
        extra = np.stack([
            feat['x_factor'],
            feat['lead_arm_angle'],
            feat['wrist_velocity'],
            feat['x_factor'] * 0.5,
            feat['lead_arm_angle'] * 0.5
        ], axis=1)  # shape (num_frames, 5)
        
        seq = np.concatenate([kp_seq, extra], axis=1)  # (num_frames, 99 + 5)
        seq_tensor = torch.tensor(seq, dtype=torch.float32).unsqueeze(0)  # (1, num_frames, 104)
        
        phase_out, _ = model(seq_tensor)
        phases_pred = torch.argmax(phase_out, dim=-1).squeeze(0).cpu().numpy()  # (num_frames,)
        
        test_phase_preds.append(phases_pred)
        
        # Optional: In progress để theo dõi
        print(f"Dự đoán phases cho test video {i+1}/{len(keypoints_test)} hoàn tất "
              f"({len(phases_pred)} frames)")

print("\nHoàn tất inference cho toàn bộ test set!")
print(f"Số video test: {len(test_phase_preds)}")

Dự đoán phases cho test video 1/10 hoàn tất (257 frames)
Dự đoán phases cho test video 2/10 hoàn tất (224 frames)
Dự đoán phases cho test video 3/10 hoàn tất (288 frames)
Dự đoán phases cho test video 4/10 hoàn tất (216 frames)
Dự đoán phases cho test video 5/10 hoàn tất (280 frames)
Dự đoán phases cho test video 6/10 hoàn tất (256 frames)
Dự đoán phases cho test video 7/10 hoàn tất (216 frames)
Dự đoán phases cho test video 8/10 hoàn tất (298 frames)
Dự đoán phases cho test video 9/10 hoàn tất (232 frames)
Dự đoán phases cho test video 10/10 hoàn tất (210 frames)

Hoàn tất inference cho toàn bộ test set!
Số video test: 10


In [ ]:
# Cell 6: Similarity Scoring + Explainable Feedback (Biomechanical + Phases)

import numpy as np

# Tính pro_reference từ các video Band 8-10 trong train set
high_quality_feats = [f for f, m in zip(train_features, train_metadata) if m['score'] >= 8]
if high_quality_feats:
    pro_reference = {
        'max_x_factor': np.mean([f['max_x_factor'] for f in high_quality_feats]),
        'max_wrist_vel': np.mean([f['max_wrist_vel'] for f in high_quality_feats]),
        'avg_sway': np.mean([f['avg_sway'] for f in high_quality_feats]),
        'impact_ratio': 0.02,  # giữ nguyên hoặc tính trung bình nếu có phase label thật
        'backswing_ratio': 0.45,
    }
    print("Đã cập nhật pro_reference từ Band 8-10 thật:")
    print(pro_reference)
# Pro reference values (dựa trên golfer chuyên nghiệp điển hình)
# Bạn có thể tinh chỉnh bằng cách lấy average từ các video Band 8-10 trong train set
pro_reference = {
    'max_x_factor': 48.0,              # Độ lệch rotation shoulder-hip (X-factor) ~45-55°
    'max_wrist_vel': 0.18,             # Tốc độ wrist cao (proxy club head speed)
    'avg_sway': 0.015,                 # Hip sway rất thấp (ổn định)
    'impact_ratio': 0.02,              # Impact thường chỉ chiếm ~1-3% frames
    'backswing_ratio': 0.45,           # Backswing chiếm ~40-50% tổng thời gian swing
    'transition_smoothness': 0.85      # Tỷ lệ velocity mượt ở top → downswing (placeholder)
}

def compute_bio_similarity(player_feat, player_phases, pro_ref):
    """
    Tính similarity score 0-1 dựa trên nhiều metrics biomechanical + phase timing
    """
    scores = {}
    
    # Rotation (X-factor)
    scores['x_factor'] = max(0, 1 - abs(player_feat['max_x_factor'] - pro_ref['max_x_factor']) / 60.0)
    
    # Wrist velocity (speed)
    scores['wrist_vel'] = min(player_feat['max_wrist_vel'] / pro_ref['max_wrist_vel'], 1.0)
    
    # Sway (stability)
    scores['sway'] = max(0, 1 - player_feat['avg_sway'] / 0.08)  # threshold 0.08 là khá cao
    
    # Phase distribution
    phase_counts = np.bincount(player_phases, minlength=8)
    total_frames = len(player_phases)
    
    # Impact ratio
    impact_ratio = phase_counts[5] / total_frames if total_frames > 0 else 0
    scores['impact'] = max(0, 1 - abs(impact_ratio - pro_ref['impact_ratio']) / 0.05)
    
    # Backswing ratio (phases 1+2+3)
    backswing_frames = phase_counts[1] + phase_counts[2] + phase_counts[3]
    backswing_ratio = backswing_frames / total_frames if total_frames > 0 else 0
    scores['backswing'] = max(0, 1 - abs(backswing_ratio - pro_ref['backswing_ratio']) / 0.3)
    
    # Overall similarity: trung bình có trọng số
    weights = {'x_factor': 0.30, 'wrist_vel': 0.25, 'sway': 0.15, 'impact': 0.15, 'backswing': 0.15}
    overall_sim = sum(scores[k] * weights[k] for k in weights)
    
    return overall_sim, scores

def generate_detailed_feedback(player_feat, player_phases, sim_score, original_score):
    """
    Tạo feedback dễ hiểu, dựa trên biomechanics và phase
    """
    issues = []
    praises = []
    
    # Rotation
    if player_feat['max_x_factor'] < 35:
        issues.append("Rotation (X-factor) thấp → thiếu sức mạnh và khoảng cách.")
    elif player_feat['max_x_factor'] > 60:
        issues.append("Rotation quá mức → dễ mất cân bằng hoặc slice.")
    else:
        praises.append("Rotation tốt.")
    
    # Stability (sway)
    if player_feat['avg_sway'] > 0.04:
        issues.append("Hip sway cao → gây mất ổn định, thường dẫn đến hook/slice.")
    else:
        praises.append("Ổn định tốt (ít sway).")
    
    # Speed / Acceleration
    if player_feat['max_wrist_vel'] < 0.10:
        issues.append("Tốc độ wrist thấp → thiếu lag và acceleration → khoảng cách ngắn.")
    else:
        praises.append("Tốc độ tốt.")
    
    # Phase timing
    phase_counts = np.bincount(player_phases, minlength=8)
    total = len(player_phases)
    
    impact_ratio = phase_counts[5] / total if total > 0 else 0
    if impact_ratio > 0.05:
        issues.append("Impact kéo → có thể early release hoặc cast.")
    elif impact_ratio < 0.005:
        issues.append("Impact khó xác định → swing có thể thiếu rõ ràng.")
    
    backswing_ratio = (phase_counts[1] + phase_counts[2] + phase_counts[3]) / total
    if backswing_ratio < 0.35:
        issues.append("Backswing ngắn → transition vội → dễ mất kiểm soát.")
    elif backswing_ratio > 0.60:
        issues.append("Backswing quá dài → dễ mất tempo.")
    
    # Tổng hợp feedback
    if sim_score > 0.85:
        overall = "Swing xuất sắc! Rất gần với biomechanics của golfer chuyên nghiệp."
    elif sim_score > 0.70:
        overall = "Swing khá tốt, chỉ cần tinh chỉnh nhỏ."
    elif sim_score > 0.50:
        overall = "Swing trung bình, có tiềm năng cải thiện rõ rệt."
    else:
        overall = "Cần luyện tập nhiều hơn để cải thiện biomechanics cơ bản."
    
    feedback_text = f"{overall}\n\n"
    if praises:
        feedback_text += "Điểm mạnh: " + "; ".join(praises) + ".\n"
    if issues:
        feedback_text += "Cần cải thiện: " + "; ".join(issues) + ".\n"
    else:
        feedback_text += "Không có lỗi lớn nào đáng kể.\n"
    
    feedback_text += f"\nSo với label gốc (band): {original_score} → Similarity AI: {sim_score:.3f}"
    
    return feedback_text

# =======================
# CHẠY PHÂN TÍCH & FEEDBACK CHO TEST SET
# =======================
print("\n" + "="*70)
print("          BIOMECHANICAL SWING ANALYSIS & FEEDBACK (Test Set)")
print("="*70 + "\n")

for i in range(len(test_features)):
    feat = test_features[i]
    phases = test_phase_preds[i]  # Từ inference ở Cell 5
    meta = test_metadata[i]
    
    sim_score, detail_scores = compute_bio_similarity(feat, phases, pro_reference)
    feedback = generate_detailed_feedback(feat, phases, sim_score, meta['score'])
    
    print(f"Test Video {i+1}/{len(test_features)}")
    print(f"  Môi trường: {meta['env']}")
    print(f"  Band chất lượng: {meta['band']}")
    print(f"  Score gốc (coach): {meta['score']}")
    print(f"  Similarity với pro (AI): {sim_score:.3f}")
    print("-"*60)
    print("Feedback chi tiết:")
    print(feedback)
    print("="*70 + "\n")

Đã cập nhật pro_reference từ Band 8-10 thật:
{'max_x_factor': 257.4653841951831, 'max_wrist_vel': 0.2380026142862595, 'avg_sway': 0.028681870130068343, 'impact_ratio': 0.02, 'backswing_ratio': 0.45}

          BIOMECHANICAL SWING ANALYSIS & FEEDBACK (Test Set)

Test Video 1/10
  Môi trường: Indoor
  Band chất lượng: Band 2-4
  Score gốc (coach): 3
  Similarity với pro (AI): 0.535
------------------------------------------------------------
Feedback chi tiết:
Swing trung bình, có tiềm năng cải thiện rõ rệt.

Điểm mạnh: Ổn định tốt (ít sway).; Tốc độ tốt..
Cần cải thiện: Rotation quá mức → dễ mất cân bằng hoặc slice.; Impact khó xác định → swing có thể thiếu rõ ràng.; Backswing ngắn → transition vội → dễ mất kiểm soát..

So với label gốc (band): 3 → Similarity AI: 0.535

Test Video 2/10
  Môi trường: Outdoor
  Band chất lượng: Band 1-2
  Score gốc (coach): 1.5
  Similarity với pro (AI): 0.548
------------------------------------------------------------
Feedback chi tiết:
Swing trung bình

In [17]:
# Evaluation so sánh similarity vs score coach
similarities = []
original_scores = []
for i in range(len(test_features)):
    sim, _ = compute_bio_similarity(test_features[i], test_phase_preds[i], pro_reference)
    similarities.append(sim)
    original_scores.append(test_metadata[i]['score'])

corr = np.corrcoef(similarities, original_scores)[0,1]
print(f"Correlation giữa AI similarity và score coach: {corr:.3f}")
# Nếu corr > 0.6 → model tốt; >0.8 → rất tốt

Correlation giữa AI similarity và score coach: -0.354


In [18]:
import pandas as pd

results = []
for i in range(len(test_features)):
    sim, _ = compute_bio_similarity(test_features[i], test_phase_preds[i], pro_reference)
    feedback = generate_detailed_feedback(test_features[i], test_phase_preds[i], sim, test_metadata[i]['score'])
    results.append({
        'video_id': i+1,
        'env': test_metadata[i]['env'],
        'band': test_metadata[i]['band'],
        'coach_score': test_metadata[i]['score'],
        'ai_similarity': round(sim, 3),
        'feedback': feedback
    })

df_results = pd.DataFrame(results)
df_results.to_csv('golf_swing_analysis_results.csv', index=False, encoding='utf-8-sig')
print("Đã export kết quả vào file: golf_swing_analysis_results.csv")

Đã export kết quả vào file: golf_swing_analysis_results.csv


In [19]:
# Cell 6: Similarity Scoring + Explainable Feedback (Cải thiện v2)

import numpy as np
import pandas as pd

# =======================
# 1. Cập nhật pro_reference từ Band 8-10 thật (ưu tiên cao nhất)
# =======================
high_quality_feats = [f for f, m in zip(train_features, train_metadata) if m['score'] >= 8]
pro_reference = {}

if high_quality_feats:
    pro_reference = {
        'max_x_factor': np.mean([f['max_x_factor'] for f in high_quality_feats]),
        'max_wrist_vel': np.mean([f['max_wrist_vel'] for f in high_quality_feats]),
        'avg_sway': np.mean([f['avg_sway'] for f in high_quality_feats]),
        'impact_ratio': 0.02,
        'backswing_ratio': 0.45,
    }
    print("Pro reference được tính từ Band 8-10 thật:")
    for k, v in pro_reference.items():
        print(f"  {k}: {v:.3f}")
else:
    # Fallback nếu chưa có đủ data cao chất lượng
    pro_reference = {
        'max_x_factor': 48.0,
        'max_wrist_vel': 0.16,      # Giảm nhẹ để dễ đạt hơn
        'avg_sway': 0.02,
        'impact_ratio': 0.015,
        'backswing_ratio': 0.48,
    }
    print("Dùng pro reference mặc định (fallback).")

# =======================
# 2. Hàm tính similarity (cải thiện trọng số + normalize)
# =======================
def compute_bio_similarity(player_feat, player_phases, pro_ref):
    scores = {}
    
    # Rotation (X-factor)
    diff_x = abs(player_feat['max_x_factor'] - pro_ref['max_x_factor'])
    scores['x_factor'] = max(0, 1 - diff_x / 70.0)  # nới lỏng threshold
    
    # Wrist velocity (speed)
    scores['wrist_vel'] = min(1.0, player_feat['max_wrist_vel'] / (pro_ref['max_wrist_vel'] * 0.85))  # 85% pro là tốt
    
    # Sway
    scores['sway'] = max(0, 1 - player_feat['avg_sway'] / 0.06)  # threshold thấp hơn
    
    # Phase timing
    phase_counts = np.bincount(player_phases, minlength=8)
    total = len(player_phases) if len(player_phases) > 0 else 1
    
    impact_ratio = phase_counts[5] / total
    scores['impact'] = max(0, 1 - abs(impact_ratio - pro_ref['impact_ratio']) / 0.04)
    
    backswing_frames = phase_counts[1] + phase_counts[2] + phase_counts[3]
    backswing_ratio = backswing_frames / total
    scores['backswing'] = max(0, 1 - abs(backswing_ratio - pro_ref['backswing_ratio']) / 0.25)
    
    # Overall similarity (trọng số mới: ưu tiên rotation & speed)
    weights = {'x_factor': 0.35, 'wrist_vel': 0.30, 'sway': 0.15, 'impact': 0.10, 'backswing': 0.10}
    overall_sim = sum(scores[k] * weights[k] for k in weights)
    
    return overall_sim, scores

# =======================
# 3. Feedback cải thiện (ngắn gọn, ưu tiên lỗi nặng, tránh lặp)
# =======================
def generate_detailed_feedback(player_feat, player_phases, sim_score, original_score):
    issues = []
    praises = []
    
    # Rotation
    if player_feat['max_x_factor'] < 32:
        issues.append("Rotation quá thấp → thiếu power và khoảng cách.")
    elif player_feat['max_x_factor'] > 62:
        issues.append("Rotation quá mức → dễ mất cân bằng/slice.")
    else:
        praises.append("Rotation ổn.")
    
    # Sway
    if player_feat['avg_sway'] > 0.045:
        issues.append("Hip sway cao → mất ổn định, dễ hook/slice.")
    else:
        praises.append("Ổn định tốt.")
    
    # Speed
    if player_feat['max_wrist_vel'] < 0.09:
        issues.append("Tốc độ wrist thấp → thiếu acceleration.")
    else:
        praises.append("Tốc độ ổn.")
    
    # Phase
    phase_counts = np.bincount(player_phases, minlength=8)
    total = len(player_phases)
    impact_ratio = phase_counts[5] / total if total > 0 else 0
    
    if impact_ratio > 0.06:
        issues.append("Impact kéo dài → early release hoặc cast.")
    elif impact_ratio < 0.005:
        issues.append("Impact khó xác định → swing thiếu rõ ràng.")
    
    backswing_ratio = (phase_counts[1] + phase_counts[2] + phase_counts[3]) / total
    if backswing_ratio < 0.38:
        issues.append("Backswing ngắn → transition vội.")
    elif backswing_ratio > 0.58:
        issues.append("Backswing dài → mất tempo.")
    
    # Tổng hợp
    if sim_score > 0.80:
        overall = "Swing rất tốt, gần pro!"
    elif sim_score > 0.65:
        overall = "Swing khá, chỉ cần chỉnh nhỏ."
    elif sim_score > 0.50:
        overall = "Swing trung bình, tiềm năng cải thiện."
    else:
        overall = "Cần luyện tập nhiều để cải thiện cơ bản."
    
    feedback_text = f"{overall}\n\n"
    if praises:
        feedback_text += f"Điểm mạnh: {'; '.join(praises)}.\n"
    if issues:
        feedback_text += f"Cần cải thiện: {'; '.join(issues[:3])}.\n"  # Giới hạn 3 lỗi chính
    feedback_text += f"\nCoach score gốc: {original_score} | AI similarity: {sim_score:.3f}"
    
    return feedback_text

# =======================
# 4. Chạy phân tích & Feedback
# =======================
print("\n" + "="*80)
print("          BIOMECHANICAL SWING ANALYSIS & FEEDBACK (Test Set - Cải thiện)")
print("="*80 + "\n")

results = []
for i in range(len(test_features)):
    feat = test_features[i]
    phases = test_phase_preds[i]
    meta = test_metadata[i]
    
    sim_score, detail_scores = compute_bio_similarity(feat, phases, pro_reference)
    feedback = generate_detailed_feedback(feat, phases, sim_score, meta['score'])
    
    print(f"Test Video {i+1}/{len(test_features)}")
    print(f"  Môi trường: {meta['env']}")
    print(f"  Band: {meta['band']} | Coach score: {meta['score']}")
    print(f"  AI Similarity: {sim_score:.3f}")
    print("-"*60)
    print("Feedback:")
    print(feedback)
    print("Chi tiết metrics:")
    for k, v in detail_scores.items():
        print(f"  {k}: {v:.3f}")
    print("="*80 + "\n")
    
    results.append({
        'video_id': i+1,
        'env': meta['env'],
        'band': meta['band'],
        'coach_score': meta['score'],
        'ai_similarity': round(sim_score, 3),
        'feedback': feedback
    })

# =======================
# 5. Export CSV + Correlation nhanh
# =======================
df_results = pd.DataFrame(results)
df_results.to_csv('golf_analysis_improved.csv', index=False, encoding='utf-8-sig')
print("Đã export kết quả vào file: golf_analysis_improved.csv")

# Correlation
sims = df_results['ai_similarity'].values
scores = df_results['coach_score'].values
corr = np.corrcoef(sims, scores)[0,1]
print(f"Correlation AI similarity vs coach score: {corr:.3f}")

Pro reference được tính từ Band 8-10 thật:
  max_x_factor: 257.465
  max_wrist_vel: 0.238
  avg_sway: 0.029
  impact_ratio: 0.020
  backswing_ratio: 0.450

          BIOMECHANICAL SWING ANALYSIS & FEEDBACK (Test Set - Cải thiện)

Test Video 1/10
  Môi trường: Indoor
  Band: Band 2-4 | Coach score: 3
  AI Similarity: 0.496
------------------------------------------------------------
Feedback:
Cần luyện tập nhiều để cải thiện cơ bản.

Điểm mạnh: Ổn định tốt.; Tốc độ ổn..
Cần cải thiện: Rotation quá mức → dễ mất cân bằng/slice.; Impact khó xác định → swing thiếu rõ ràng.; Backswing ngắn → transition vội..

Coach score gốc: 3 | AI similarity: 0.496
Chi tiết metrics:
  x_factor: 0.000
  wrist_vel: 1.000
  sway: 0.667
  impact: 0.500
  backswing: 0.461

Test Video 2/10
  Môi trường: Outdoor
  Band: Band 1-2 | Coach score: 1.5
  AI Similarity: 0.490
------------------------------------------------------------
Feedback:
Cần luyện tập nhiều để cải thiện cơ bản.

Điểm mạnh: Ổn định tốt.; Tốc độ 

In [20]:
# Cell 6: Similarity Scoring + Feedback (Cải thiện v3 - Adaptive & Scale tốt hơn)

import numpy as np
import pandas as pd

# =======================
# 1. Pro reference adaptive (mean + median, fallback an toàn)
# =======================
high_quality_feats = [f for f, m in zip(train_features, train_metadata) if m['score'] >= 8]
pro_reference = {}

if high_quality_feats:
    # Dùng median để giảm ảnh hưởng outlier
    pro_reference = {
        'max_x_factor': np.median([f['max_x_factor'] for f in high_quality_feats]),
        'max_wrist_vel': np.median([f['max_wrist_vel'] for f in high_quality_feats]),
        'avg_sway': np.median([f['avg_sway'] for f in high_quality_feats]),
        'impact_ratio': 0.018,      # nới lỏng hơn
        'backswing_ratio': 0.47,
    }
    print("Pro reference adaptive từ Band 8-10 (median):")
    for k, v in pro_reference.items():
        print(f"  {k}: {v:.3f}")
else:
    pro_reference = {
        'max_x_factor': 45.0,
        'max_wrist_vel': 0.14,
        'avg_sway': 0.025,
        'impact_ratio': 0.015,
        'backswing_ratio': 0.48,
    }
    print("Fallback pro reference (median-style).")

# =======================
# 2. Similarity (nới lỏng + sigmoid scale để band cao đạt cao hơn)
# =======================
def compute_bio_similarity(player_feat, player_phases, pro_ref):
    scores = {}
    
    # Rotation
    diff_x = abs(player_feat['max_x_factor'] - pro_ref['max_x_factor'])
    scores['x_factor'] = 1 / (1 + np.exp(5 * (diff_x / 70 - 0.5)))  # sigmoid nới
    
    # Wrist velocity (scale lên nếu gần hoặc vượt pro)
    vel_ratio = player_feat['max_wrist_vel'] / pro_ref['max_wrist_vel']
    scores['wrist_vel'] = 1 / (1 + np.exp(-8 * (vel_ratio - 0.7)))  # thưởng nếu gần 70-100%
    
    # Sway
    scores['sway'] = 1 / (1 + np.exp(10 * (player_feat['avg_sway'] - 0.04)))
    
    # Phase
    phase_counts = np.bincount(player_phases, minlength=8)
    total = max(len(player_phases), 1)
    
    impact_ratio = phase_counts[5] / total
    scores['impact'] = 1 / (1 + np.exp(20 * (abs(impact_ratio - pro_ref['impact_ratio']) - 0.02)))
    
    backswing_frames = phase_counts[1] + phase_counts[2] + phase_counts[3]
    backswing_ratio = backswing_frames / total
    scores['backswing'] = 1 / (1 + np.exp(10 * (abs(backswing_ratio - pro_ref['backswing_ratio']) - 0.2)))
    
    # Tổng similarity (trọng số + scale lên 0.6-0.9 cho band tốt)
    weights = {'x_factor': 0.30, 'wrist_vel': 0.30, 'sway': 0.20, 'impact': 0.10, 'backswing': 0.10}
    raw_sim = sum(scores[k] * weights[k] for k in weights)
    overall_sim = 0.4 + raw_sim * 0.6  # scale lên để band cao đạt ~0.7-0.85
    
    return overall_sim, scores

# =======================
# 3. Feedback (ngắn hơn, gợi ý luyện tập cụ thể)
# =======================
def generate_detailed_feedback(player_feat, player_phases, sim_score, original_score):
    issues = []
    praises = []
    suggestions = []
    
    if player_feat['max_x_factor'] < 32:
        issues.append("Rotation thấp")
        suggestions.append("Luyện drill coil shoulder-hip (ví dụ: pause tại top).")
    elif player_feat['max_x_factor'] > 62:
        issues.append("Rotation quá mức")
        suggestions.append("Giảm over-rotation bằng drill giữ head steady.")
    
    if player_feat['avg_sway'] > 0.045:
        issues.append("Hip sway cao")
        suggestions.append("Drill feet together hoặc alignment stick trên hông.")
    else:
        praises.append("Ổn định tốt.")
    
    if player_feat['max_wrist_vel'] < 0.09:
        issues.append("Tốc độ wrist thấp")
        suggestions.append("Luyện lag drill với towel hoặc pump drill.")
    else:
        praises.append("Tốc độ ổn.")
    
    phase_counts = np.bincount(player_phases, minlength=8)
    total = len(player_phases)
    impact_ratio = phase_counts[5] / total if total > 0 else 0
    
    if impact_ratio > 0.06:
        issues.append("Impact kéo")
        suggestions.append("Luyện impact bag hoặc slow-motion drill.")
    elif impact_ratio < 0.005:
        issues.append("Impact khó xác định")
        suggestions.append("Kiểm tra setup và tempo cơ bản.")
    
    backswing_ratio = (phase_counts[1] + phase_counts[2] + phase_counts[3]) / total
    if backswing_ratio < 0.38:
        issues.append("Backswing ngắn")
        suggestions.append("Luyện backswing full với mirror check.")
    
    # Tổng hợp
    if sim_score > 0.80:
        overall = "Swing rất tốt, gần pro!"
    elif sim_score > 0.65:
        overall = "Swing khá, chỉ cần chỉnh nhỏ."
    elif sim_score > 0.50:
        overall = "Swing trung bình, tiềm năng cải thiện."
    else:
        overall = "Cần luyện tập nhiều để cải thiện cơ bản."
    
    feedback_text = f"{overall}\n\n"
    if praises:
        feedback_text += f"Điểm mạnh: {'; '.join(praises)}.\n"
    if issues:
        feedback_text += f"Lỗi chính: {'; '.join(issues[:2])}.\n"
        feedback_text += f"Gợi ý luyện: {'; '.join(suggestions[:2])}.\n"
    feedback_text += f"\nCoach score: {original_score} | AI similarity: {sim_score:.3f}"
    
    return feedback_text

# =======================
# 4. Chạy & Export
# =======================
print("\n" + "="*80)
print("          BIOMECHANICAL SWING ANALYSIS - CẢI THIỆN v3")
print("="*80 + "\n")

results = []
for i in range(len(test_features)):
    feat = test_features[i]
    phases = test_phase_preds[i]
    meta = test_metadata[i]
    
    sim_score, detail_scores = compute_bio_similarity(feat, phases, pro_reference)
    feedback = generate_detailed_feedback(feat, phases, sim_score, meta['score'])
    
    print(f"Video {i+1}/{len(test_features)} - Band {meta['band']} (score {meta['score']})")
    print(f"AI Similarity: {sim_score:.3f}")
    print(feedback)
    print("Metrics chi tiết:")
    for k, v in detail_scores.items():
        print(f"  {k}: {v:.3f}")
    print("-"*80)
    
    results.append({
        'video_id': i+1,
        'env': meta['env'],
        'band': meta['band'],
        'coach_score': meta['score'],
        'ai_similarity': round(sim_score, 3),
        'feedback': feedback
    })

df = pd.DataFrame(results)
df.to_csv('golf_analysis_v3.csv', index=False, encoding='utf-8-sig')
print("\nĐã export: golf_analysis_v3.csv")

# Correlation
sims = df['ai_similarity'].values
scores = df['coach_score'].values
corr = np.corrcoef(sims, scores)[0,1]
print(f"Correlation AI vs Coach: {corr:.3f}")

Pro reference adaptive từ Band 8-10 (median):
  max_x_factor: 276.481
  max_wrist_vel: 0.130
  avg_sway: 0.030
  impact_ratio: 0.018
  backswing_ratio: 0.470

          BIOMECHANICAL SWING ANALYSIS - CẢI THIỆN v3

Video 1/10 - Band Band 2-4 (score 3)
AI Similarity: 0.720
Swing khá, chỉ cần chỉnh nhỏ.

Điểm mạnh: Ổn định tốt.; Tốc độ ổn..
Lỗi chính: Rotation quá mức; Impact khó xác định.
Gợi ý luyện: Giảm over-rotation bằng drill giữ head steady.; Kiểm tra setup và tempo cơ bản..

Coach score: 3 | AI similarity: 0.720
Metrics chi tiết:
  x_factor: 0.038
  wrist_vel: 1.000
  sway: 0.550
  impact: 0.510
  backswing: 0.611
--------------------------------------------------------------------------------
Video 2/10 - Band Band 1-2 (score 1.5)
AI Similarity: 0.725
Swing khá, chỉ cần chỉnh nhỏ.

Điểm mạnh: Ổn định tốt.; Tốc độ ổn..
Lỗi chính: Rotation quá mức; Impact khó xác định.
Gợi ý luyện: Giảm over-rotation bằng drill giữ head steady.; Kiểm tra setup và tempo cơ bản..

Coach score: 1.5 | 

In [21]:
torch.save(model.state_dict(), 'golf_phase_model.pth')